In [168]:
import sys
import os
from IPython.core.interactiveshell import InteractiveShell
from platform import python_version
print(python_version())
InteractiveShell.ast_node_interactivity = "all"
print("My working directory:\n" + os.getcwd())

3.6.8
My working directory:
/home/dsxuser/work


In [169]:
#!pip install geopy

In [170]:
#!pip install geopy

In [171]:
import numpy as np
import pandas as pd

print('Libraries imported.')

Libraries imported.


### Step 0: Read table from Wikipedia

In [172]:
temp = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [173]:
temp[0].head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h3>Step 1:</h3>

<ol>
    <li>Take the table we need from result</li>
</ol>

<hr>

In [174]:
df_PostCodeM = pd.DataFrame(temp[0])
#df_PostCodeM.columns = df_PostCodeM.iloc[0]
#df_PostCodeM.drop([0], inplace = True)
#df_PostCodeM.reset_index(drop=True, inplace=True)
df_PostCodeM.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


<h3>Step 2: drop rows with 'Not assigned' Borough</h3>

In [175]:
df_PostCodeM_adj = df_PostCodeM[df_PostCodeM['Borough'] != 'Not assigned']
df_PostCodeM_adj.reset_index(drop=True, inplace=True)
df_PostCodeM_adj.head(10)
df_PostCodeM_adj.shape

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


(210, 3)

<h3>Step 3: Replace the 'Not assigned' Neighborhood with the value of borough</h3>

In [176]:
for i in range(df_PostCodeM_adj.shape[0]):
    if df_PostCodeM_adj.iloc[i]['Neighborhood'] == 'Not assigned':
        df_PostCodeM_adj.iloc[i]['Neighborhood'] = df_PostCodeM_adj.iloc[i]['Borough']

In [177]:
(df_PostCodeM_adj['Neighborhood']=='Not assigned').unique()
df_PostCodeM_adj.head(10)

array([False])

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<h3>Step 4: merge neighborhoods with the same postal code</h3>

In [178]:
#a function to create a dataframe with merged neighborhoods with the same postal code
def merge_neighbor(temp):
    a = str()
    for i in range(temp.shape[0]):
        a = a + temp.iloc[i]['Neighborhood'] +', '
    result_list = [temp.iloc[0,0], temp.iloc[0,1], a[0:-2]]
    result_df = pd.DataFrame(result_list, index=['Postcode', 'Borough', 'Neighborhood']).T
    return result_df

In [179]:
#get all the postal code (no repetition)
Postcode = df_PostCodeM_adj['Postcode'].unique()

#merge neighborhoods for each postal code
for item in Postcode:
    temp = df_PostCodeM_adj[df_PostCodeM_adj['Postcode'] == item]
    #temp
    result_df = merge_neighbor(temp)
    df_PostCodeM_adj = df_PostCodeM_adj[df_PostCodeM_adj['Postcode'] != item]
    df_PostCodeM_adj = df_PostCodeM_adj.append(result_df, ignore_index=True, sort=False)
    #df_PostCodeM_adj.shape

In [180]:
#show 11 rows and shape
df_PostCodeM_adj.head(5)
df_PostCodeM_adj.shape

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


(103, 3)

### Step 5: get the geographical coordinates of the neighborhoods

In [181]:
#It seems pgeocode is a better and more reliable package
#!pip install pgeocode
from pgeocode import Nominatim

In [182]:
#use PyPl pgeocode package find geographical coordinates
nomi = Nominatim('CA')
Latitude = []
Longitude = []

for item in df_PostCodeM_adj['Postcode']:
    Geo_Coord = nomi.query_postal_code(item)
    Latitude.append(Geo_Coord.loc['latitude'])
    Longitude.append(Geo_Coord.loc['longitude'])

In [183]:
#put those geo-coordinates into the dataframe
Latitude_df = pd.DataFrame(Latitude)
Longitude_df = pd.DataFrame(Longitude)
df_PostCodeM_adj['Latitude'] = Latitude_df
df_PostCodeM_adj['Longitude'] = Longitude_df
df_PostCodeM_adj.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889


### check to see if there is something missing

In [184]:
df_anynull = df_PostCodeM_adj.isnull()
df_anynull.describe()
df_anynull['Latitude'].value_counts()
df_anynull['Longitude'].value_counts()
df_PostCodeM_adj[df_anynull['Latitude'] == True]

,Postcode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103,103
unique,1,1,1,2,2
top,False,False,False,False,False
freq,103,103,103,102,102


False    102
True       1
Name: Latitude, dtype: int64

False    102
True       1
Name: Longitude, dtype: int64

,Postcode,Borough,Neighborhood,Latitude,Longitude
76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


### Since there is a row missing value, I will use the geo-coordinates from the csv file provided to get 'M7R' value

In [185]:
#read from the CSV file
#!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data
df = pd.read_csv('Geospatial_Coordinates.csv')
df[df['Postal Code'] == 'M7R']

,Postal Code,Latitude,Longitude
86,M7R,43.636966,-79.615819


In [186]:
df_PostCodeM_adj.iloc[76,3] = df[df['Postal Code'] == 'M7R'].iloc[0,1]
df_PostCodeM_adj.iloc[76,4] = df[df['Postal Code'] == 'M7R'].iloc[0,2]

In [187]:
#check again to make sure there is no null
df_anynull = df_PostCodeM_adj.isnull()
df_anynull.describe()
df_anynull['Latitude'].value_counts()
df_anynull['Longitude'].value_counts()
df_PostCodeM_adj[df_anynull['Latitude'] == True]

,Postcode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103,103
unique,1,1,1,1,1
top,False,False,False,False,False
freq,103,103,103,103,103


False    103
Name: Latitude, dtype: int64

False    103
Name: Longitude, dtype: int64

,Postcode,Borough,Neighborhood,Latitude,Longitude


In [188]:
df_PostCodeM_adj.head(11)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889
5,M9A,Queen's Park,Queen's Park,43.6662,-79.5282
6,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
7,M3B,North York,Don Mills North,43.7450,-79.3590
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7063,-79.3094
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3783
